# 全体の流れ

## 分析の前準備
- BigQueryから収集したデータの抽出
- データを扱いやすく整形
- 統計情報を確認
- データの可視化
- データの特徴
    - 外気温は比較的一定
    - CPU温度はよく変化する
    
## 分析の実施

CPU温度が外気温を越えて熱くならないようにコントロールしたい。そのためにどうデータを扱うかのサンプルを確認する。ここで作ったモデルを実際にプロダクション環境に組み込めることを想定して作りましょう。

# 分析の前準備
## BigQueryからのデータ抽出とデータ整形

In [ ]:
%%bq query -n requests
SELECT datetime, cpu_temperature, temperature
FROM `soracom_handson.raspi_env`
order by datetime asc 

In [ ]:
import google.datalab.bigquery as bq
import pandas as pd

df_from_bq = requests.execute(output_options=bq.QueryOutput.dataframe()).result()

In [ ]:
# データの確認
df_from_bq

In [ ]:
# 文字列型でデータが取得されているので変換
df_from_bq['datetime'] = pd.to_datetime(df_from_bq['datetime'])
df_from_bq['cpu_temperature'] = df_from_bq['cpu_temperature'].astype('float')
df_from_bq['temperature'] = df_from_bq['temperature'].astype('float')

# 時系列データ化する
df = df_from_bq[['cpu_temperature', 'temperature']]
df.index = df_from_bq['datetime'].values

In [ ]:
# データの確認
df

## 統計情報の確認

In [ ]:
df.describe()

## 時系列データとして可視化する

In [ ]:
df.plot(y=['cpu_temperature', 'temperature'], figsize=(16,4), alpha=0.5)

## 散布図として可視化する

In [ ]:
df.plot(kind='scatter', x='cpu_temperature', y='temperature', c='cpu_temperature', cmap='winter')

## 相関係数を確認する

In [ ]:
# 散布図から定性的に確認できた相関をさらに定量的に確認する
import numpy as np

np.corrcoef(df['cpu_temperature'], df['temperature'])

# 分析の実施
## Step1：最新のCPU温度と外気温を比較する
時間順にソートされた最新のデータを使って、CPU温度と外気温を比較する方法

In [ ]:
df.tail(1)['load_1min'] > df.tail(1)['load_15min']

### この方法の問題点
温度の大小関係が少しでも変わるとすぐに判定が変わってしまう。もう少しなだらかに判定するロジックとしたい

## Step2：最新からN個のデータを使ってCPU温度と外気温を比較する
時間順にソートされたN個のデータを使って、CPU温度が外気温より大きくなっている場合の数をカウントする方法

In [ ]:
N = 10
threashold = 8
df_N = df.tail(N)
sum(df_N['load_1min'] > df_N['load_15min']) > threashold

### この方法の問題点
Step1よりは確実に良いロジックとなっているが、引き続き、閾値の設定がややシビアに思える。もう少しなだからな判定ロジックとしたい。

## Step3：移動平均使って比較する

In [ ]:
# 10項移動平均
df.rolling(window=20, center=False).mean()

# 10項移動平均の最新のものを比較材料に使う
moving_average = df.rolling(window=10, center=False).mean().tail(1)

moving_average['load_1min'] > moving_average['load_15min']